# Deployed by Nandini Lokesh Reddy

## Data - Overview

In [ ]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("qwedsacf/grade-school-math-instructions")
print(ds)


README.md:   0%|          | 0.00/852 [00:00<?, ?B/s]

(…)-00000-of-00001-3f5d416810641542.parquet:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8792 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE', 'SOURCE'],
        num_rows: 8792
    })
})


In [ ]:
print(ds["train"][0])

{'INSTRUCTION': 'This math problem has got me stumped: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nCan you show me the way?', 'RESPONSE': 'Natalia sold 48/2 = 24 clips in May.\nNatalia sold 48+24 = 72 clips altogether in April and May.', 'SOURCE': 'grade-school-math'}


# converting the data into JSON

In [3]:
import json
import pandas as pd
from datasets import load_dataset

def prepare_math_data(dataset_name="qwedsacf/grade-school-math-instructions"):
    # Load dataset from Hugging Face
    dataset = load_dataset(dataset_name)

    # Convert to fine-tuning format
    def format_example(example):
        return {
            "messages": [
                {"role": "system", "content": "You are a helpful math tutor."},
                {"role": "user", "content": example["Instruction"]},
                {"role": "assistant", "content": example["Response"]}
            ]
        }

    # Process training data
    training_data = [format_example(item) for item in dataset["train"]]

    # Save formatted data
    with open('math_training.jsonl', 'w') as f:
        for item in training_data:
            f.write(json.dumps(item) + '\n')

    # Create validation split
    df = pd.DataFrame(training_data)
    train_df = df.sample(frac=0.8, random_state=42)
    val_df = df.drop(train_df.index)

    return train_df, val_df

def analyze_data(train_df, val_df):
    total_examples = len(train_df) + len(val_df)
    avg_instruction_len = train_df['messages'].apply(
        lambda x: len(x[1]['content'])
    ).mean()
    avg_response_len = train_df['messages'].apply(
        lambda x: len(x[2]['content'])
    ).mean()

    return {
        "total_examples": total_examples,
        "training_examples": len(train_df),
        "validation_examples": len(val_df),
        "avg_instruction_len": avg_instruction_len,
        "avg_response_len": avg_response_len
    }

In [4]:
pip install transformers datasets torch accelerate

In [5]:
pip install openai datasets

In [ ]:
import json
import openai
from datasets import load_dataset
from typing import List, Dict
import os

def prepare_training_data(dataset_name: str = "qwedsacf/grade-school-math-instructions") -> str:
    """Prepare training data in OpenAI format"""
    dataset = load_dataset(dataset_name)

    training_data = []
    for item in dataset["train"]:
        training_example = {
            "messages": [
                {"role": "system", "content": "You are a helpful math tutor."},
                {"role": "user", "content": item["INSTRUCTION"]},
                {"role": "assistant", "content": item["RESPONSE"]}
            ]
        }
        training_data.append(training_example)

    # Save to JSONL file
    output_file = "math_training.jsonl"
    with open(output_file, "w") as f:
        for item in training_data:
            f.write(json.dumps(item) + "\n")

    return output_file

def start_finetuning(
    file_path: str,
    model: str = "gpt-3.5-turbo",
    openai_api_key: str = None
) -> str:
    """Start fine-tuning job"""
    if openai_api_key:
        openai.api_key = openai_api_key

    # Upload file
    file_upload = openai.files.create(
        file=open(file_path, "rb"),
        purpose="fine-tune"
    )

    # Create fine-tuning job
    job = openai.fine_tuning.jobs.create(
        training_file=file_upload.id,
        model=model,
        hyperparameters={
            "n_epochs": 3
        }
    )

    return job.id

def main():
    # Set your OpenAI API key
    api_key = "your-openai-key"
    if not api_key:
        raise ValueError("Please set OPENAI_API_KEY environment variable")

    # Prepare data and start fine-tuning
    file_path = prepare_training_data()
    job_id = start_finetuning(file_path, openai_api_key=api_key)
    print(f"Fine-tuning job started. Job ID: {job_id}")

if __name__ == "__main__":
    main()

## monitoring

In [ ]:
import openai
import time

def monitor_job(job_id):
    while True:
        job = openai.fine_tuning.jobs.retrieve(job_id)
        print(f"Status: {job.status}")

        if job.status in ["succeeded", "failed"]:
            print("Final status:", job.__dict__)
            break

        time.sleep(60)  # Check every minute

# Monitor job
monitor_job("JOBID")

## ready to use the model

In [ ]:
job = openai.fine_tuning.jobs.retrieve("JOBID")
print(f"Fine-tuned model ID: {job.fine_tuned_model}")

In [ ]:
response = openai.chat.completions.create(
    model=job.fine_tuned_model,
    messages=[{"role": "user", "content": "Your math question"}]
)

## inference

In [14]:
import openai

def test_math_model(model_id):
    # Test questions
    test_questions = [
        "If John has 15 marbles and loses 7, how many does he have left?",
        "A store sells shirts for $25 each. If they sell 12 shirts, what's the total revenue?",
        "Sara has 48 candies and gives half to her sister. How many candies does Sara have now?"
    ]

    for question in test_questions:
        response = openai.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": question}],
            temperature=0.7
        )
        print(f"\nQ: {question}")
        print(f"A: {response.choices[0].message.content}\n")
        print("-" * 50)

# Get model ID and test
job = openai.fine_tuning.jobs.retrieve("YOUR-JOBID")
test_math_model(job.fine_tuned_model)


Q: If John has 15 marbles and loses 7, how many does he have left?
A: 15 - 7 = 8
John has 8 marbles left.

--------------------------------------------------

Q: A store sells shirts for $25 each. If they sell 12 shirts, what's the total revenue?
A: Total revenue is $25 * 12 = $300.

--------------------------------------------------

Q: Sara has 48 candies and gives half to her sister. How many candies does Sara have now?
A: Sara gave half of her 48 candies to her sister which is 48 / 2 = 24 candies.
So, Sara is left with 48 - 24 = 24 candies.

--------------------------------------------------
